In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [5]:
from rs_datasets import MovieLens

data = MovieLens("25m")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828



items


,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance



tags


,user_id,item_id,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598



links


,item_id,imdb_id,tmdb_id
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [6]:
from sponge_bob_magic.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "relevance": "rating",
        "timestamp": "timestamp"
    }
)

In [7]:
from sponge_bob_magic.models import LightFMWrap

model = LightFMWrap()

In [8]:
from sponge_bob_magic.splitters import UserSplitter

user_random_splitter = UserSplitter(
    item_test_size=1,
    user_test_size=10000,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=1234
)

In [9]:
from sponge_bob_magic.scenarios import MainScenario
from sponge_bob_magic.metrics import NDCG, HitRate

scenario = MainScenario(
    splitter=user_random_splitter,
    recommender=model,
    criterion=HitRate,
    metrics={
        NDCG: [10, 5, 1],
        HitRate: [10, 5, 1],
    }
)

In [10]:
from sponge_bob_magic.session_handler import State
from pyspark.sql.functions import split

genres = (
    State().session.createDataFrame(data.items[["item_id", "genres"]])
    .select(
        "item_id",
        split("genres", "\|").alias("genres")
    )
)

In [11]:
genres.show()

+-------+--------------------+
|item_id|              genres|
+-------+--------------------+
|      1|[Adventure, Anima...|
|      2|[Adventure, Child...|
|      3|   [Comedy, Romance]|
|      4|[Comedy, Drama, R...|
|      5|            [Comedy]|
|      6|[Action, Crime, T...|
|      7|   [Comedy, Romance]|
|      8|[Adventure, Child...|
|      9|            [Action]|
|     10|[Action, Adventur...|
|     11|[Comedy, Drama, R...|
|     12|    [Comedy, Horror]|
|     13|[Adventure, Anima...|
|     14|             [Drama]|
|     15|[Action, Adventur...|
|     16|      [Crime, Drama]|
|     17|    [Drama, Romance]|
|     18|            [Comedy]|
|     19|            [Comedy]|
|     20|[Action, Comedy, ...|
+-------+--------------------+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import explode

genres_list = (
    genres.select(explode("genres").alias("genre"))
    .distinct().filter('genre <> "(no genres listed)"')
    .toPandas()["genre"].tolist()
)

/home/shminke-ba/projects/sponge-bob-magic/venv/lib/python3.6/site-packages/pyarrow/util.py:39: FutureWarning: pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead
  warnings.warn(msg, FutureWarning)


In [13]:
genres_list

['Documentary',
 'Fantasy',
 'IMAX',
 'Adventure',
 'War',
 'Animation',
 'Comedy',
 'Thriller',
 'Film-Noir',
 'Crime',
 'Sci-Fi',
 'Musical',
 'Mystery',
 'Drama',
 'Horror',
 'Western',
 'Romance',
 'Children',
 'Action']

In [14]:
from pyspark.sql.functions import col, lit, array_contains
from pyspark.sql.types import IntegerType

item_features = genres
for genre in genres_list:
    item_features = item_features.withColumn(
        genre,
        array_contains(col("genres"), genre).astype(IntegerType())
    )
item_features = item_features.drop("genres").cache()
item_features.count()

62423

In [18]:
scenario.research(
    {"no_components": [128]},
    log,
    k=10,
    n_trials=1,
    item_features=item_features
)

14-May-20 12:09:38, sponge_bob_magic, DEBUG: Деление лога на обучающую и тестовую выборку
14-May-20 12:09:43, sponge_bob_magic, DEBUG: Длина трейна и теста: 24990095 9998
14-May-20 12:09:44, sponge_bob_magic, DEBUG: Количество пользователей в трейне и тесте: 162541, 9998
14-May-20 12:09:44, sponge_bob_magic, DEBUG: Количество объектов в трейне и тесте: 59045, 2849
14-May-20 12:09:44, sponge_bob_magic, DEBUG: Инициализация метрик
14-May-20 12:09:44, sponge_bob_magic, DEBUG: Обучение и предсказание дополнительной модели
14-May-20 12:09:44, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
14-May-20 12:09:49, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
14-May-20 12:10:14, sponge_bob_magic, DEBUG: Оптимизация параметров
14-May-20 12:10:14, sponge_bob_magic, DEBUG: Количество попыток: 1
14-May-20 12:10:27, sponge_bob_magic, DEBUG: -- Второй фит модели в оптимизации
14-May-20 12:10:27, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
14-May-20 1

{'no_components': 128}

In [ ]:
scenario.experiment.results

,HitRate@10,HitRate@1,HitRate@5,NDCG@1,NDCG@5,NDCG@10
"LightFMWrap(no_components=128, loss=bpr, random_state=None)",0.181536,0.04751,0.123925,0.04751,0.085882,0.104383


In [20]:
scenario.research(
    {"no_components": [128]},
    log,
    k=10,
    n_trials=1,
)

14-May-20 13:41:14, sponge_bob_magic, DEBUG: Деление лога на обучающую и тестовую выборку
14-May-20 13:41:15, sponge_bob_magic, DEBUG: Длина трейна и теста: 24990095 9998
14-May-20 13:41:16, sponge_bob_magic, DEBUG: Количество пользователей в трейне и тесте: 162541, 9998
14-May-20 13:41:17, sponge_bob_magic, DEBUG: Количество объектов в трейне и тесте: 59045, 2849
14-May-20 13:41:17, sponge_bob_magic, DEBUG: Инициализация метрик
14-May-20 13:41:17, sponge_bob_magic, DEBUG: Обучение и предсказание дополнительной модели
14-May-20 13:41:17, sponge_bob_magic, DEBUG: Предварительная стадия обучения (pre-fit)
14-May-20 13:41:20, sponge_bob_magic, DEBUG: Основная стадия обучения (fit)
/home/shminke-ba/projects/sponge-bob-magic/venv/lib/python3.6/site-packages/pyarrow/util.py:39: FutureWarning: pyarrow.open_stream is deprecated as of 0.17.0, please use pyarrow.ipc.open_stream instead
  warnings.warn(msg, FutureWarning)
14-May-20 13:41:45, sponge_bob_magic, DEBUG: Оптимизация параметров
14-May-

{'no_components': 128}

In [21]:
scenario.experiment.results

,HitRate@10,HitRate@1,HitRate@5,NDCG@1,NDCG@5,NDCG@10
"LightFMWrap(no_components=128, loss=bpr, random_state=None)",0.20184,0.052711,0.136927,0.052711,0.095318,0.116217
